# Apex - PROD

### Account Summary

In [7]:
Import-Module AWS.Tools.Common, AWS.Tools.DynamoDBv2, AWS.Tools.RDS
Import-Module '/Users/conrad.gauntlett/WorkArea/Repos/DBA_MISC/PowerShell/Modules/DevOpsToolkit'

$Profiles   = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -match 'DevOps-Apex-PROD' }
$Regions    = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"
$Summary    = Get-DoAWSDBInformation $Profiles $Regions
$Summary    | Format-Table -AutoSize


Account   AccountID    Region    RDSInstances DynamoDBTables
-------   ---------    ------    ------------ --------------
Apex-PROD 463854377715 eu-west-2            0              8



Error: Command failed: SubmitCode: Import-Module AWS.Tools.Common, AWS.Tools.DynamoDB ...

### DynamoDB Tables

In [ ]:
$ddbTables = Get-DoDDBTableInformation $Profiles $Regions 
$ddbTables | Format-Table -AutoSize

#### Backups

In [ ]:
$BackupsList    = Get-DDBBackupList -ProfileName $Profiles.ProfileName 
$Columns        = 'TableName', 'BackupName', 'BackupCreationDateTime',   'BackupStatus'#, 'BackupSizeBytes', 'BackupType' 
$BackupsList | Select-Object -Property $Columns | Format-Table -AutoSize

In [ ]:
# $Backup = New-DDBBackup -TableName 'prod-apex_translations' -ProfileName DevOps-Apex-PROD -BackupName 'prod-apex_translations_BACKUP4'
# $Backup

In [ ]:
$Bk = Get-DDBBackup -BackupArn $Backup.BackupArn -ProfileName DevOps-Apex-PROD
$Bk.BackupDetails